In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd /content/drive/MyDrive/Llama3_Finetuning/LLama3_Finetuning

In [ ]:
!git clone https://github.com/bala1802/LLama3_Finetuning.git

In [ ]:
import config
import prompts
import dataset
import llama_model
import inference
import peft_model
import train

#### Model and Tokenizer Initialization

In [ ]:
model, tokenizer = llama_model.initialize_model_tokenizer()

#### Parameter Efficient Finetuning Model

In [ ]:
model = peft_model.get_peft_model(model=model)

#### Data Loader

In [ ]:
from datasets import load_dataset
from prompts import ALPACA_PROMPT

EOS_TOKEN = tokenizer.eos_token
def format_prompt(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = ALPACA_PROMPT.format(instruction, input, output) + EOS_TOKEN # without this token generation goes on forever!
        texts.append(text)
    return { "text" : texts, }

def extract_dataset():
    dataset = load_dataset(config.DATASET, split = "train")
    dataset = dataset.map(format_prompt, batched = True,)
    return dataset

dataset = extract_dataset()

#### Model Finetuning

In [ ]:
trainer = train.train(model=model, tokenizer=tokenizer, dataset=dataset)

#### Model Inferencing

In [ ]:
from unsloth import FastLanguageModel

FastLanguageModel.for_inference(model)

inference.generate(model=model,
                   tokenizer=tokenizer,
                   instruction="Convert these binary numbers to decimal.",
                   input="1010, 1101, 1111")